# A Quick Introduction to Optuna

This Jupyter notebook goes through the basic usage of Optuna.

- Install Optuna
- Write a training algorithm that involves hyperparameters
  - Read train/valid data
  - Define and train model
  - Evaluate model
- Use Optuna to tune the hyperparameters (hyperparameter optimization, HPO)
- Visualize HPO

## Install `optuna`

Optuna can be installed via `pip` or `conda`.

In [1]:
!pip install --quiet optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.4 MB/s eta 0:00:00


In [2]:
import optuna

optuna.__version__

'3.5.0'

## Optimize Hyperparameters

### Define a simple scikit-learn model

We start with a simple random forest model to classify flowers in the Iris dataset. We define a function called `objective` that encapsulates the whole training process and outputs the accuracy of the model.

In [3]:
import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection

def objective():
    iris = sklearn.datasets.load_iris()  # Prepare the data.

    clf = sklearn.ensemble.RandomForestClassifier(
        n_estimators=5, max_depth=3)  # Define the model.

    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3).mean()  # Train and evaluate the model.

print('Accuracy: {}'.format(objective()))

Accuracy: 0.94


### Optimize hyperparameters of the model

The hyperparameters of the above algorithm are `n_estimators` and `max_depth` for which we can try different values to see if the model accuracy can be improved. The `objective` function is modified to accept a trial object. This trial has several methods for sampling hyperparameters. We create a study to run the hyperparameter optimization and finally read the best hyperparameters.

In [4]:
import optuna

def objective(trial):
    iris = sklearn.datasets.load_iris()

    n_estimators = trial.suggest_int('n_estimators', 2, 20)
    max_depth = int(trial.suggest_float('max_depth', 1, 32, log=True))

    clf = sklearn.ensemble.RandomForestClassifier(
        n_estimators=n_estimators, max_depth=max_depth)

    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3).mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2024-01-21 17:47:14,086] A new study created in memory with name: no-name-1340177f-89be-413d-9a0a-645fa5268475
[I 2024-01-21 17:47:14,132] Trial 0 finished with value: 0.96 and parameters: {'n_estimators': 5, 'max_depth': 3.5185850403464967}. Best is trial 0 with value: 0.96.
[I 2024-01-21 17:47:14,390] Trial 1 finished with value: 0.9666666666666667 and parameters: {'n_estimators': 19, 'max_depth': 14.871222116587484}. Best is trial 1 with value: 0.9666666666666667.
[I 2024-01-21 17:47:14,519] Trial 2 finished with value: 0.7600000000000001 and parameters: {'n_estimators': 17, 'max_depth': 1.1998309034381798}. Best is trial 1 with value: 0.9666666666666667.
[I 2024-01-21 17:47:14,580] Trial 3 finished with value: 0.9466666666666667 and parameters: {'n_estimators': 9, 'max_depth': 6.575562874824405}. Best is trial 1 with value: 0.9666666666666667.
[I 2024-01-21 17:47:14,625] Trial 4 finished with value: 0.9533333333333333 and parameters: {'n_estimators': 3, 'max_depth': 2.4412683303

Accuracy: 0.9733333333333333
Best hyperparameters: {'n_estimators': 20, 'max_depth': 4.634025186672749}


It is possible to condition hyperparameters using Python `if` statements. We can for instance include another classifier, a support vector machine, in our HPO and define hyperparameters specific to the random forest model and the support vector machine.

In [5]:
import sklearn.svm

def objective(trial):
    iris = sklearn.datasets.load_iris()

    classifier = trial.suggest_categorical('classifier', ['RandomForest', 'SVC'])

    if classifier == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 2, 20)
        max_depth = int(trial.suggest_float('max_depth', 1, 32, log=True))

        clf = sklearn.ensemble.RandomForestClassifier(
            n_estimators=n_estimators, max_depth=max_depth)
    else:
        c = trial.suggest_float('svc_c', 1e-10, 1e10, log=True)

        clf = sklearn.svm.SVC(C=c, gamma='auto')

    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3).mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2024-01-21 17:48:48,966] A new study created in memory with name: no-name-3b16e275-a35f-4f5d-accd-1207204b8f31
[I 2024-01-21 17:48:49,005] Trial 0 finished with value: 0.7466666666666667 and parameters: {'classifier': 'RandomForest', 'n_estimators': 3, 'max_depth': 1.1933758941385677}. Best is trial 0 with value: 0.7466666666666667.
[I 2024-01-21 17:48:49,096] Trial 1 finished with value: 0.96 and parameters: {'classifier': 'RandomForest', 'n_estimators': 14, 'max_depth': 4.989883064088834}. Best is trial 1 with value: 0.96.
[I 2024-01-21 17:48:49,215] Trial 2 finished with value: 0.9466666666666667 and parameters: {'classifier': 'RandomForest', 'n_estimators': 13, 'max_depth': 16.774717401695543}. Best is trial 1 with value: 0.96.
[I 2024-01-21 17:48:49,346] Trial 3 finished with value: 0.96 and parameters: {'classifier': 'RandomForest', 'n_estimators': 15, 'max_depth': 5.816846354172031}. Best is trial 1 with value: 0.96.
[I 2024-01-21 17:48:49,431] Trial 4 finished with value: 0.

Accuracy: 0.9866666666666667
Best hyperparameters: {'classifier': 'SVC', 'svc_c': 4.917272706903988}


### Plotting the study

Plotting the optimization history of the study.

In [6]:
optuna.visualization.plot_optimization_history(study)

Plotting the accuracies for each hyperparameter for each trial.

In [7]:
optuna.visualization.plot_slice(study)

Plotting the accuracy surface for the hyperparameters involved in the random forest model.

In [8]:
optuna.visualization.plot_contour(study, params=['n_estimators', 'max_depth'])